In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [2]:
COMBINED_DATA_FPATH = '../input/textdatanumericdatacombined/DATA_WITH_TXT.h5'
SAMPLE_SUBMISSION_FPATH = '../input/competitive-data-science-predict-future-sales/sample_submission.csv'
TRAINED_MODEL_FPATH = '../input/alldatamodel/all_data_model.bin'

In [3]:
from catboost import CatBoostRegressor
regressor_kwargs = {
    'depth': 14,
    'iterations':2500,
    'random_seed':0,
    'learning_rate':0.04,
    'od_type':'Iter',
    'od_wait':300,
    'task_type':'GPU',
                   }

model = CatBoostRegressor(**regressor_kwargs)

In [4]:
model.load_model(TRAINED_MODEL_FPATH)
print(model.get_params())
print(model.random_seed_)
print(model.learning_rate_)

{'task_type': 'GPU', 'random_seed': 0, 'od_type': 'Iter', 'od_wait': 300, 'loss_function': 'RMSE', 'depth': 14, 'learning_rate': 0.04, 'iterations': 1500, 'verbose': 100, 'logging_level': 'Verbose'}
0
0.03999999911


In [5]:
test_X_df =pd.read_hdf('../input/textdatanumericdatacombined/DATA_WITH_TXT.h5','X_test')
test_X_df.to_hdf('JustTestData.h5','X_test')

In [6]:
from catboost import Pool
sample_submissions_df = pd.read_csv(SAMPLE_SUBMISSION_FPATH, index_col=0)
test_X_df = test_X_df.loc[sample_submissions_df.index,:]
test_prediction = model.predict(Pool(data=test_X_df))
test_prediction[test_prediction > 20] = 20
test_prediction[test_prediction < 0]  = 0
sample_submissions_df['item_cnt_month'] = test_prediction

In [7]:
sample_submissions_df.to_csv('submission.csv')

In [8]:
sample_submissions_df.head()

,item_cnt_month
ID,
0,0.724397
1,0.957017
2,1.065805
3,0.467499
4,5.895402
